# Model - 1

The model's objective, neglecting the entirety of the variables, to regulate the production of the packages $ t_i $ in order to maximize the collection $ \ sum_i price_ix_i $ being $ x_i $ the quantity (not integer) of boxes of the type $ i $ .
Obviously, since each box contains different types of tarallo inside, the constraint according to which $ \ sum_j type_j * x_i \ leq maxquant_j $ must be respected. We then want to impose a constraint on the number of individual boxes with respect to the total, that is, we want to impose the constraint $ x_i \ leq perc_i \ sum_i (x_i) $. So the model is:

$$ \max \sum_i prezzo_ix_i \\
\sum_i kind_j \cdot x_i \leq maxquant_j \\
x_i \leq perc_i\sum_i(x_i) \\
x_i \geq 0 \ \ \ \forall i
$$

In [1]:
import pyomo.environ as pe
import pandas as pd

In [2]:
#getting the data from an Excel target file
path = 'Compito_E_1864353_dati.xlsx'
df = pd.read_excel(path, index_col=0)
df

,Taralli al finocchietto,Taralli al peperoncino,Taralli allo zenzero,Prezzo di vendita,Percentuale massima
t1,20,3,12,3.0,0.35
t2,10,17,0,12.0,0.45
t3,7,5,18,16.0,0.40
t4,10,15,5,11.0,0.38
t5,10,20,10,10.0,0.55
Quantità limite,6000,5100,3200,NaN,NaN


In [3]:
#obtaining model's sets
#different kinds of commercial boxes (every row but the last)
S = df.index[:-1]
#different kinds of tarallo (an another italian type of bread...) (first three columns of the dataframe)
T = df.columns[0:3]
#prices
prezzi = df.loc[S, 'Prezzo di vendita']
#maximal commercial percentage allowed to be sold
perc = df.loc[S, 'Percentuale massima']
# max number of tarallos for each kind of tarallo
qlim = df.loc['Quantità limite',T]

# the total number of tarallos for any kind of tarallo and box
nt = df.loc[S,T]

# Model -  1

In [5]:
#defying the model
mdl = pe.ConcreteModel('Modello Esame - Compito E')
mdl.x = pe.Var(S, within = pe.NonNegativeReals)

mdl.obj = pe.Objective(expr = sum(prezzi[s]*mdl.x[s] for s in S), 
                       #volendo massimizzare
                        sense = -1)

#not going above the maximum quantity available for each kind of tarallo
def vinc_tar(m, t):
    return sum(mdl.x[s]*nt.at[s,t] for s in S) <= qlim[t]
mdl.c_disptar = pe.Constraint(T, rule = vinc_tar)

#not going above the maximum percentage of each kind of box
def perc_mas(m, s):
    return m.x[s] <= perc[s]*sum(m.x[s] for s in S)
mdl.c_percmas = pe.Constraint(S, rule = perc_mas)

#getting a solver
solver = pe.SolverFactory('glpk')
#solving the model
solver.solve(mdl)

{'Problem': [{'Name': 'unknown', 'Lower bound': 5592.69854824936, 'Upper bound': 5592.69854824936, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 6, 'Number of nonzeros': 40, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.01636195182800293}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

# Modello punto 1

In [6]:
#displaying the solutions
res = pd.DataFrame(index = S, columns=['Quantità'])
for _ in mdl.x:
    res.at[_] = mdl.x[_]()
display(res)
print('Total revenue equals {:.3f}'.format(mdl.obj()))

,Quantità
t1,0.000000
t2,188.877028
t3,157.365500
t4,73.484202
t5,0.000000


Incasso complessivo pari a 5592.699


# Duality - 2
As Optimizazion Theory have well shown, the kind of tarallo to prefer is the one which presents the highest shadow price. To obtain that information is crucial to get the dual of the problem of interest.

In [7]:
######################################################################definisco il modello 
mdl = pe.ConcreteModel('Modello Esame - Compito E')
mdl.x = pe.Var(S, within = pe.NonNegativeReals)

mdl.obj = pe.Objective(expr = sum(prezzi[s]*mdl.x[s] for s in S), 
                       #volendo massimizzare
                        sense = -1)


def vinc_tar(m, t):
    return sum(mdl.x[s]*nt.at[s,t] for s in S) <= qlim[t]
mdl.c_disptar = pe.Constraint(T, rule = vinc_tar)


def perc_mas(m, s):
    return m.x[s] <= perc[s]*sum(m.x[s] for s in S)
mdl.c_percmas = pe.Constraint(S, rule = perc_mas)
########################################################################

#obtaing and solving the dual to the problem
mdl.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
solver.solve(mdl)

{'Problem': [{'Name': 'unknown', 'Lower bound': 5592.69854824936, 'Upper bound': 5592.69854824936, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 6, 'Number of nonzeros': 40, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.018479108810424805}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
#accessing each shadow price
for h in mdl.component_objects(pe.Constraint, active=True):
    print ("Constraint",h)
    for index in h:
        print ('Il vincolo {} ha un prezzo ombra pari a {}'.format(index, mdl.dual[h[index]]))

Constraint c_disptar
Il vincolo Taralli allo zenzero ha un prezzo ombra pari a 0.828351836037575
Il vincolo Taralli al peperoncino ha un prezzo ombra pari a 0.576857386848847
Il vincolo Taralli al finocchietto ha un prezzo ombra pari a 0.0
Constraint c_percmas
Il vincolo t2 ha un prezzo ombra pari a 3.98804440649017
Il vincolo t5 ha un prezzo ombra pari a 0.0
Il vincolo t1 ha un prezzo ombra pari a 0.0
Il vincolo t4 ha un prezzo ombra pari a 0.0
Il vincolo t3 ha un prezzo ombra pari a 0.0


To automatically return the identifier of the kind of tarallo which has the highest shadow price is possible to use:

In [9]:
import operator
prezzi_omb = {}
for r in T:
    prezzi_omb[r] = mdl.dual[mdl.c_disptar[r]]
TO_INC = max(prezzi_omb.items(), key=operator.itemgetter(1))[0]
print('Better increasing disponibilty of {} . They present a shadow price of {:.2f} €'.
     format(TO_INC, mdl.dual[mdl.c_disptar[TO_INC]]))

Conviene incrementare la disponibilità di Taralli allo zenzero che presentano un prezzo ombra pari a 0.83 €


# Binary Variable - 3

Taking into account the fact that when storing tarallos there is the necessity to pay for this storage, it is introduced a binary variable to model that in order that the model also considers what is best in terms of revenue. This is done via use of binary variable $\delta$

In [10]:
### third version
mdl = pe.ConcreteModel('Modello Esame - Compito E')
mdl.x = pe.Var(S, within = pe.NonNegativeReals)
mdl.delta = pe.Var(within = pe.Binary)
mdl.obj = pe.Objective(expr = sum(prezzi[s]*mdl.x[s] for s in S) - 50*mdl.delta, 
                       #maximize
                        sense = -1)


def vinc_tar(m, t):
    return sum(mdl.x[s]*nt.at[s,t] for s in S) <= qlim[t]
mdl.c_disptar = pe.Constraint(T, rule = vinc_tar)


def perc_mas(m, s):
    return m.x[s] <= perc[s]*sum(m.x[s] for s in S)
mdl.c_percmas = pe.Constraint(S, rule = perc_mas)

#linking binary variable being one when actually producing t2 tarallo's type. 
bigM = 1e6
mdl.c_log = pe.Constraint(expr = mdl.x['t2'] <= mdl.delta*bigM)

solver.solve(mdl)

{'Problem': [{'Name': 'unknown', 'Lower bound': 5542.69854824936, 'Upper bound': 5542.69854824936, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 7, 'Number of nonzeros': 42, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '3', 'Number of created subproblems': '3'}}, 'Error rc': 0, 'Time': 0.021603822708129883}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
res_p3 = pd.DataFrame(index = S, columns=['Quantità'])
for _ in mdl.x:
    res_p3.at[_] = mdl.x[_]()
display(res_p3)
print('Incasso complessivo pari a {:.3f}'.format(mdl.obj()))

,Quantità
t1,0.000000
t2,188.877028
t3,157.365500
t4,73.484202
t5,0.000000


Incasso complessivo pari a 5542.699


# Iterating solve process - 4

The problem is now solved several times by increasing the value of the availability of 'Taralli allo zenzero' in each iteration until this increase in availability no longer results in an increase in the value of the revenue. This increase takes place without considering the commercial modification taken into proper account in point 3.

It can be observed that the increase in the availability of 'Taralli al peperoncino' logically entails an increase in the value of the objective function, given the fact that the value of the dual variable associated with the primal constraint for compliance with the availability (limit quantity) of 'Taralli allo zenzero'is equal to $ 0.83> 0 $

In [13]:
#setting a bound
qlim['Taralli allo zenzero'] = 5100

qincr = 500

################################################### defyning the model as at point 1
mdl = pe.ConcreteModel('Modello Esame - Compito E')
mdl.x = pe.Var(S, within = pe.NonNegativeReals)

mdl.obj = pe.Objective(expr = sum(prezzi[s]*mdl.x[s] for s in S), 
                       
                        sense = -1)


def vinc_tar(m, t):
    return sum(mdl.x[s]*nt.at[s,t] for s in S) <= qlim[t]
mdl.c_disptar = pe.Constraint(T, rule = vinc_tar)


def perc_mas(m, s):
    return m.x[s] <= perc[s]*sum(m.x[s] for s in S)
mdl.c_percmas = pe.Constraint(S, rule = perc_mas)
###################################################

################################################### defyning again the dual 
mdl.dual = pe.Suffix(direction = pe.Suffix.IMPORT)
solver.solve(mdl)

while mdl.dual[mdl.c_disptar['Taralli al peperoncino']]>0:
    qlim['Taralli al peperoncino'] += 500

    #getting rid of the availability constraint
    mdl.del_component(mdl.c_disptar)
    #getting rid of indexes
    mdl.del_component(mdl.c_disptar_index)
    #getting new value having increased the general maximal availability value
    mdl.c_disptar = mdl.c_disptar = pe.Constraint(T, rule = vinc_tar)
    
    #solve
    pe.SolverFactory('glpk').solve(mdl)
    
    #accedo ai risultati per ogni iterazione (per capire se tutto funziona come dovrebbe)
    res_p4 = pd.DataFrame(index = S, columns=['Quantità'])
    for _ in mdl.x:
        res_p4.at[_] = mdl.x[_]()

#display results
display(res_p4)

print('The highest value to which it is justified to increase' +
      'availability is {}'.format(qlim['Taralli allo zenzero']))

print("Maximal revenue having obtained the maximal availability increase {:.3f}".format(mdl.obj()))

,Quantità
t1,0.000000
t2,285.135624
t3,112.115732
t4,236.383363
t5,0.000000


Il massimo valore conveniente di disponibilità è pari a 9100.0
Incasso al massimo valore conveniente di disponibilità è pari a 7815.696
